In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

%run utils.ipynb
df = testimon_fraud()

**training dataset** - a set of examples used to fit the parametres
**validation dataset** - provides an unbiased evaluation of a model fit while tuning the models' hyperparameters.  This dataset can be used for regularization by early stopping 
**test dataset** is a dataset used to provide an evaluation of a final model fit on the training dataset.

In [ ]:
# Generate test set
y = df['isFraud'].values # npdarray
x = df.drop('isFraud',axis=1).values # npdarray

from sklearn.model_selection import train_test_split

# Assigning random_state makes sure that although the process is pseudo-random, 
# it will always return the same split, which makes our work more reproducible
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Generate validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


## Oversampling
Oversampling is useful when the dataset has imbalanced.  In this case, the fraction of fraudulent transactions is 